# Continuous Control Project

I'll begin this project by implementing a straight DDPG solution and see how that works.  Depending on the amount of time it takes, I hope to follow up with additional layers of sophistication.

-----

## Solution 1 - Simple DDPG

The code below is based on the Udacity instructor-provided code from the financial markets lesson.  I have modified it somewhat to adapt to the robot arm problem for this project.

### Set up the Unity reacher environment for a single arm

In [ ]:
!pip -q install ./python
print("Python install done.")

from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='Reacher_Linux/Reacher.x86_64')
print("Environment defined.")

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print(brain)

In [ ]:
##### Set up the environment

import time
import numpy as np
import random
from collections import namedtuple, deque
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from ddpg_agent import DdpgAgent

%matplotlib inline


In [ ]:
WINNING_SCORE = 30.0 #average over 100 consecutive episodes

def train(agent, env, run_name="UNDEF", max_episodes=20, max_time_steps=1000, break_in=512, sleeping=False):

    # Initialize simulation environment
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]
    #env_info = env.reset(train_mode=True)[brain_name]
    #state_size = len(env_info.vector_observations[0])
    #action_size = brain.vector_action_space_size

    scores = []
    recent_scores = deque(maxlen=100)
    start_time = time.perf_counter()
    starting_point = 0

    # loop on episodes
    for e in range(starting_point, max_episodes):
        
        # Reset the enviroment
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0 #total score for this episode

        # loop over time steps
        for i in range(max_time_steps):

            # Predict the best action for the current state.
            action = agent.act(state)

            # get the new state & reward based on this action
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]

            # update the agent with this new info
            agent.step(state, action, reward, next_state, done)

            # roll over new state
            state = next_state
            score += reward
            if done:
                break

        # determine epoch duration and estimate remaining time
        current_time = time.perf_counter() #was time.clock()
        avg_duration = (current_time - start_time) / (e - starting_point + 1) / 60.0 #minutes
        remaining_time_minutes = (starting_point + max_episodes - e - 1) * avg_duration
        rem_time = remaining_time_minutes / 60.0
        time_est_msg = "{:4.1f} hr rem.".format(rem_time)

        # update score bookkeeping, report status and decide if training is complete
        scores.append(score)
        recent_scores.append(score)
        avg_score = np.mean(recent_scores)
        print('\rEpisode {}\tAverage Score: {:.2f}, avg {:.1f} episodes/min'
              .format(e, avg_score, 1.0/avg_duration), end="")
        if e > 0  and  e % 50 == 0:
            torch.save(agent.actor_local.state_dict(),  '{}_checkpointa_{:d}.pt'.format(run_name, e))
            torch.save(agent.critic_local.state_dict(), '{}_checkpointc_{:d}.pt'.format(run_name, e))
            print('\rEpisode {}\tAverage Score: {:.2f}\t{}             '.format(e, avg_score, time_est_msg))

        if sleeping:
            if e % 50 < 5:
                time.sleep(1) #allow time to view the Unity window

        if e > 100  and  avg_score >= WINNING_SCORE:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(e, avg_score))
            torch.save(agent.actor_local.state_dict(),  '{}_checkpointa.pt'.format(run_name))
            torch.save(agent.critic_local.state_dict(), '{}_checkpointc.pt'.format(run_name))
            break

    return scores

In [ ]:
## Train the model & observe its progress

RUN_NAME = 'D'
BATCH = 128
DECAY = 0.99999
LEARN_EVERY = 25
LEARN_ITER  = 40
a = DdpgAgent(33, 4, random_seed=0, batch_size=BATCH, noise_decay=DECAY, 
              learn_every=LEARN_EVERY, learn_iter=LEARN_ITER)

scores = train(a, env, run_name=RUN_NAME, max_episodes=1000, max_time_steps=1000, 
               break_in=BATCH, sleeping=True)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()


In [ ]:
#env.close()

### Run a pre-trained agent

Use the code below to load an agent from checkpoint files and run it in inference mode.

**Note:**  It uses an environment and some other variables set up above, so don't close the environment in the previous cell, and don't plan to run this cell by itself.

In [ ]:
# enter the names of the two checkpoint files you want to run here:
ACTOR =  "B/checkpointa_950.pt"
#CRITIC = "B/checkpointc_100.pt"

#BATCH = 128
#DECAY = 0.99999
#LEARN_EVERY = 25
#LEARN_ITER  = 40

inference_agent = DdpgAgent(33, 4, random_seed=19004, actor_file=ACTOR)

for episode in range(3):
    env_info = env.reset(train_mode=False)[brain_name]   
    state = env_info.vector_observations[0]
    score = 0.0
    print("Episode {} ".format(episode), end='')
    
    for time_step in range(500):
        action = inference_agent.act(state)
        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        inference_agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            print("Done signal.")
            break
            
    print("total score: {:.2f}".format(score))
    time.sleep(3)


In [ ]:
env.close()